In [153]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.utils import plot_model

In [154]:
temp = pd.read_csv("obesity.csv")
columns = ['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC', 'CALC', 'MTRANS']
for c in columns:
  dummies = pd.get_dummies(temp[c], prefix=(str(c)+"_"))
  temp = pd.concat([temp, dummies], axis=1)
temp = temp.drop(columns=columns)


d = ['Insufficient_Weight',
       'Normal_Weight', 'Overweight_Level_I', 'Overweight_Level_II', 'Obesity_Type_I',
       'Obesity_Type_II', 'Obesity_Type_III']
ordinal_encoder = OrdinalEncoder(categories=[d])
temp['NObeyesdad'] = ordinal_encoder.fit_transform(temp[['NObeyesdad']])
y_xgb = temp['NObeyesdad']

temp = temp.drop('NObeyesdad', axis=1)
X_train, X_test, y_train, y_test = train_test_split(temp, y_xgb, test_size=0.2)

In [155]:
print(y_xgb)

0       1.0
1       1.0
2       1.0
3       2.0
4       3.0
       ... 
2106    6.0
2107    6.0
2108    6.0
2109    6.0
2110    6.0
Name: NObeyesdad, Length: 2111, dtype: float64


In [156]:
print(temp['NCP'].value_counts())

3.000000    1203
1.000000     199
4.000000      69
2.776840       2
3.985442       2
            ... 
3.054899       1
3.118013       1
3.335876       1
3.205009       1
1.089048       1
Name: NCP, Length: 635, dtype: int64


In [157]:
model = XGBClassifier(objective='multi:softprob')

In [158]:
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [159]:
y_pred = model.predict(X_test)
y_pred

array([5, 5, 1, 2, 6, 4, 1, 6, 5, 4, 6, 4, 5, 1, 4, 1, 4, 6, 0, 4, 6, 6,
       2, 1, 3, 3, 1, 6, 5, 4, 0, 5, 5, 2, 6, 0, 1, 3, 5, 2, 0, 5, 5, 4,
       2, 1, 2, 3, 5, 4, 2, 0, 0, 6, 6, 5, 0, 5, 0, 0, 4, 3, 0, 4, 6, 0,
       2, 4, 5, 6, 2, 2, 1, 6, 6, 0, 2, 5, 2, 2, 0, 4, 0, 5, 4, 6, 1, 1,
       1, 2, 1, 4, 6, 4, 2, 5, 6, 6, 6, 1, 3, 1, 4, 5, 5, 4, 0, 5, 1, 0,
       1, 0, 0, 3, 4, 1, 1, 3, 3, 5, 1, 5, 6, 1, 4, 0, 6, 2, 4, 6, 6, 6,
       5, 0, 2, 6, 5, 6, 4, 6, 2, 5, 6, 4, 2, 2, 4, 5, 1, 0, 4, 5, 2, 6,
       5, 2, 1, 3, 1, 1, 4, 4, 1, 0, 1, 4, 4, 0, 0, 6, 6, 6, 3, 1, 4, 3,
       0, 0, 2, 6, 5, 0, 3, 1, 4, 1, 6, 2, 6, 1, 0, 4, 4, 6, 3, 3, 3, 4,
       2, 5, 2, 3, 5, 3, 1, 0, 1, 1, 5, 1, 4, 6, 4, 3, 4, 3, 0, 3, 3, 1,
       1, 0, 0, 0, 4, 4, 3, 1, 5, 1, 2, 1, 6, 3, 2, 6, 4, 6, 0, 6, 3, 0,
       3, 6, 5, 3, 4, 6, 0, 6, 2, 3, 1, 5, 5, 1, 3, 2, 5, 4, 2, 2, 4, 4,
       4, 2, 6, 6, 5, 4, 4, 0, 2, 2, 4, 4, 2, 5, 2, 4, 1, 3, 2, 2, 1, 4,
       2, 3, 1, 2, 5, 6, 5, 4, 6, 5, 5, 6, 2, 1, 6,

In [160]:
y_pred_proba = model.predict_proba(X_test)
print(y_pred_proba)

[[1.72310865e-05 5.53249174e-06 2.16621083e-05 ... 1.44281279e-04
  9.99719083e-01 4.97278379e-05]
 [4.20253127e-05 3.05604226e-05 8.07557662e-05 ... 6.54488336e-04
  9.98797774e-01 1.40098549e-04]
 [5.32459759e-04 9.97135282e-01 1.42154587e-03 ... 2.79420725e-04
  1.22398371e-04 1.58629133e-04]
 ...
 [1.05261584e-04 1.41694676e-03 5.01377683e-04 ... 2.03758755e-04
  7.15866481e-05 3.45485227e-04]
 [2.59969620e-05 1.19787082e-05 1.29629952e-05 ... 8.17685577e-05
  2.86146242e-04 9.99531388e-01]
 [3.11004769e-05 1.85960580e-05 4.33594760e-05 ... 9.98718143e-01
  1.48747247e-04 4.04362218e-05]]


In [167]:
accuracy = model.score(X_test, y_test)
print("Accuracy: %.4f%%" % (accuracy * 100.0))

Accuracy: 97.6359%


In [162]:
#XGB Cross Validation - https://machinelearningmastery.com/evaluate-gradient-boosting-models-xgboost-python/
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
def createXGB():
  model = XGBClassifier(objective='multi:softprob', enable_categorical=True)
  return model
def train_evaluateXGB(model, xtrain, ytrain, xtest, ytest):
  model.fit(xtrain, ytrain)
  accuracy = model.score(xtest, ytest)
  return accuracy
kFold = StratifiedKFold(n_splits = 10)
i=0
sum=0
for train, test in kFold.split(temp,y_xgb):
  print("Current Iteration:",i)
  modelxgb = None
  modelxgb = createXGB()

  acc = train_evaluateXGB(modelxgb,temp.loc[[x for x in train]],y_xgb.loc[[x for x in train]],temp.loc[[x for x in test]],y_xgb.loc[[x for x in test]])
  i+=1
  sum += acc

print("Mean accuracy:", (sum/i))
#Mean accuracy: 0.96499150496289

Current Iteration: 0


Current Iteration: 1
Current Iteration: 2
Current Iteration: 3
Current Iteration: 4
Current Iteration: 5
Current Iteration: 6
Current Iteration: 7
Current Iteration: 8
Current Iteration: 9
Mean accuracy: 0.96499150496289


In [163]:
import pickle

In [164]:
filename = 'xgb_model.sav'
pickle.dump(model, open(filename, 'wb'))


In [165]:
xgb_model = pickle.load(open('xgb_model.sav', 'rb'))

In [166]:
pip install xgboost==1.3.3

^C
Note: you may need to restart the kernel to use updated packages.
